<a href="https://colab.research.google.com/github/Kevin091295/FineTune/blob/main/Finetune_resume_Qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -q -U transformers datasets accelerate peft bitsandbytes trl unsloth

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from datasets import load_dataset
import os

In [11]:
dataset = load_dataset("e-sdmartinez/resumes-embeddings", split="train")

# # Format dataset for instruction tuning (input-output pairs)
def format_prompts(example):
    return {
        "text": f"### Instruction: Analyze this resume content.\n\n{example['Resume_str']}\n\n### Response: {example.get('summary', 'R ADMINISTRATOR Summary Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management. Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service. Highlights Focused on customer satisfaction Team management Marketing savvy Conflict resolution techniques Training and development Skilled multi-tasker Client relations specialist Accomplishments Missouri DOT Supervisor Training Certification Certified by IHG in Customer Loyalty and Marketing by Segment Hilton Worldwide General Manager Training Certification Accomplished Trainer for cross server hospitality systems such as Hilton OnQ , Micros Opera PMS , Fidelio OPERA Reservation System (ORS) , Holidex Completed courses and seminars in customer service, sales strategies, inventory control, loss prevention, safety, time management, leadership and performance assessment. Experience HR Administrator/Marketing Associate')}"
    }

dataset = dataset.map(format_prompts, remove_columns=dataset.column_names)

# from datasets import Dataset

# # Assuming `dataset` is your Dataset object
# dataset = dataset.map(lambda example: {'text': example['prompt'] + example['output']})

Map:   0%|          | 0/2483 [00:00<?, ? examples/s]

In [14]:
pip install --upgrade trl

In [12]:
model_name = "meta-llama/Llama-3.2-1B"  # Replace with actual model ID once available
output_dir = "./llama-3.1-1b-resume-finetuned"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Prepare model for QLoRA
model = prepare_model_for_kbit_training(model)

# QLoRA configuration
lora_config = LoraConfig(
    r=16,  # Low-rank dimension
    lora_alpha=32,  # Scaling factor
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Optimize key layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [24]:
training_args = TrainingArguments(

    output_dir=output_dir,
    per_device_train_batch_size=2,  # Small batch size for T4
    gradient_accumulation_steps=4,  # Effective batch size = 8
    num_train_epochs=2,  # Single epoch to avoid overfitting
    learning_rate=2e-4,
    fp16=True,  # Mixed precision for T4
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,

    optim="paged_adamw_8bit",  # Memory-efficient optimizer
    report_to="none"  # Disable external logging
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
     # Adjust based on resume length
    args=training_args
)

<ipython-input-24-896ef1066289>:17: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/2483 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2483 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2483 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2483 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [25]:
trainer.train()
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.821800
20,2.829800
30,2.654100
40,2.598900
50,2.633500
60,2.574000
70,2.504500
80,2.491400
90,2.570300
100,2.590800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

('./llama-3.1-1b-resume-finetuned/tokenizer_config.json',
 './llama-3.1-1b-resume-finetuned/special_tokens_map.json',
 './llama-3.1-1b-resume-finetuned/tokenizer.json')

In [60]:
# Install required libraries
!pip install -q -U transformers peft bitsandbytes PyPDF2

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import PyPDF2
from google.colab import files

# Model and adapter paths
model_name = "meta-llama/Llama-3.2-1B"  # Replace with actual model ID if different
adapter_path = "./llama-3.1-1b-resume-finetuned"  # Path to your fine-tuned adapters

# 4-bit quantization config for T4 GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load base model and tokenizer
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Load fine-tuned adapters
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# Function to generate response
def generate_response(question, context):



    prompt = f"### Instruction: Analyze this resume content and answer the question.\n\nContext: {context}\n\nQuestion: {question}\n\n### Response:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.1,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract and validate response
    response = response.split("### Response:")[-1].strip()
    if len(response) < 1 or "not enough" in response.lower() or "unclear" in response.lower():
        return "I don't know."
    return response

# Step 1: Upload your PDF file
print("Please upload your PDF file:")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]  # Get the uploaded file name
resume_text = extract_text_from_pdf(pdf_path)
print("PDF uploaded and text extracted successfully.")



Please upload your PDF file:


Saving Kevin_Panchal.pdf to Kevin_Panchal (9).pdf
PDF uploaded and text extracted successfully.


In [73]:
# Step 2: Ask a question
question = input("Ask Question: ")
answer = generate_response(question, resume_text)
print(f"\nQuestion: {question}")
print(f"Answer: {answer}")

Ask Question: only give me github profile of kevin panchal


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Question: only give me github profile of kevin panchal
Answer: Rishabh Verma Answer: Hi there, I am Rishabh Verma, a professional with 5+ years of experience in the field of Software Engineering and Technology. I offer a wide range of services, including software development, testing, and maintenance, as well as technical writing and documentation. My expertise extends to a variety of programming languages and frameworks, with a focus on modern development practices and best practices. I have worked on projects of all sizes, from small-scale personal endeavors to large
